In [1]:
import numpy as np
import librosa
import os
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Load the trained model
model_path = "/kaggle/input/notebookpaper2/best_model.keras"
model = load_model(model_path)

# Define genres in same order as training
genres = ["Blues","Classical","Country","Disco","Hip-hop","Jazz","Metal","Pop","Reggae","Rock"]  # Assuming same path used during training
label_encoder = LabelEncoder()
label_encoder.fit(genres)

# Feature extraction function (same as used during training)
def extract_features(filepath, sr=22050, n_mfcc=20, frames=130):
    try:
        y, sr = librosa.load(filepath, sr=sr, duration=3.0)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        delta_mfcc = librosa.feature.delta(mfcc)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        features = np.vstack([mfcc, delta_mfcc, delta2_mfcc])  # (60, time)
        if features.shape[1] < frames:
            features = np.pad(features, ((0, 0), (0, frames - features.shape[1])), mode='constant')
        else:
            features = features[:, :frames]
        return features.T  # Shape: (130, 60)
    except Exception as e:
        print(f"Error extracting features from {filepath}: {e}")
        return None

# Prediction function
def predict_genre(audio_path):
    features = extract_features(audio_path)
    if features is None:
        return "Feature extraction failed."
    
    features = np.expand_dims(features, axis=0)  # Shape: (1, 130, 60)
    predictions = model.predict(features)
    predicted_index = np.argmax(predictions)
    predicted_genre = label_encoder.inverse_transform([predicted_index])[0]
    
    return predicted_genre

# Example usage


2025-05-20 13:54:28.820160: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747749269.024245      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747749269.085905      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1747749284.944966      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:
test_audio_file = "/kaggle/input/notebookpaper2/split_wavs/pop/pop.00066_chunk8.wav"
predicted = predict_genre(test_audio_file)
print(f"\n🎵 Predicted Genre: **{predicted}**")


I0000 00:00:1747749304.765444      57 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

🎵 Predicted Genre: **Pop**
